# Using  WW3 models to generate $\delta_{rms}$ for any location on land

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>")) 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr"
import os
import pandas as pd
import matplotlib.pyplot as plt
from netCDF4 import Dataset as netcdf_dataset
import netCDF4
import numpy as np
from scipy import interpolate
import scipy.integrate
from obspy.geodetics.base import gps2dist_azimuth
from obspy import UTCDateTime
from obspy.geodetics.base import kilometer2degrees, locations2degrees
import numpy.matlib


import warnings
warnings.filterwarnings('ignore')


from cartopy import config
import cartopy.crs as ccrs
os.environ["CARTOPY_USER_BACKGROUNDS"] = "BG/"

from os.path import normpath
from ww32seismo import *

# Loading the settings

The basic configuration setting are stored on the config.yml file, in working directory.

If needed, their values can be hardcoded in the cell below

An important parameter to replace is the "work_dir" on line 13.

We will start with the station UCC in Brussels and then, if time allows it, we will follow with station KNDR in Greece.

In [ ]:
import dynamic_yaml
with open("config.yml", 'r') as f:
    configs = dynamic_yaml.load(f)

target = configs.params.station
target_lat = configs.params.station_lat
target_lon = configs.params.station_lon
rhos = configs.params.rhos
beta = configs.params.beta
Rg = configs.params.Rg
Q = configs.params.Q
Re = 4.0e7/(2*np.pi)
depth_file = configs.files.depth


In [ ]:
#  needed files: Rayleigh_source.txt, depth file, and at least the models with coastal reflection /!\
dataset = netcdf_dataset(r"{}".format(normpath(depth_file)))
dpt = pd.DataFrame(np.asarray(dataset.variables["dpt"])[50,:,:], columns=dataset.variables["longitude"], index=dataset.variables["latitude"])
dpt[dpt==-32767] *= 0.0
dpt[dpt<=0.0] = 0.0

if not os.path.isdir("DATA"):
    os.mkdir("DATA")
if not os.path.isdir("FIGURES"):
    os.mkdir("FIGURES")    
if not os.path.isdir("DATA/Q"):
    os.mkdirs("DATA/Q")

# Plot depth and distance

In [ ]:
from ww32seismo import *
lats, lons, distance_df = get_distance(configs, dataset, dpt, plot=True)

In [ ]:
df = pd.read_csv(r"{}".format(configs.files.noise_source_term), header=None, delim_whitespace=True, index_col=0)
df.index *= np.pi
df = df.fillna(0.0)
C_base = (df[:8]**2).sum(axis=1)
C_base.at[C_base.index[-1]+0.01] = 0.0
C_base.at[-1.0] = 0.0
C_base.at[20.0] = 0.0
C_base = C_base.sort_index()

Cf = interpolate.interp1d(C_base.index, C_base)

# Reading the P2L file

In [ ]:
month = 10
beta = configs.params.beta
CgR = configs.params.Rg
target = configs.params.station
factor1, alpha, alpha2 = alpha_distance(configs,Re, distance_df)
fn = os.path.join(configs.files.p2l_dir,"LOPS_WW3-GLOB-30M_2021{}_p2l.nc".format(month))
fname = r"{}".format(fn)
dataset = netcdf_dataset(fname)
lats = dataset.variables['latitude'][:]
lons = dataset.variables['longitude'][:]
times = dataset.variables['time']
times = netCDF4.num2date(times[:],times.units)
freqs = dataset.variables['f'][:] # ocean wave frequency

The fit of the simulated ground-motion to real observations is affected mainly by the Q value and the reflection coefficient.
The reflection is typically paremetrized during the generation of the oceanic model and won't be explored here.
The Q value is by far the most important as for high Qs, seismic sources from a vast area of the ocean contribute to the seismic record, while a low Q means that sources are concentrated in the vicinity of the seismic station

In [ ]:
Qf = np.ones(len(freqs)) * Q
plt.figure()
plt.title("Q vs Frequency")
plt.plot(freqs, Qf)
plt.xlabel("Frequency")
plt.ylabel("Q")
plt.show()

A constant Q in frequency such as the one above can lead to a poor shape of the synthetic noise spectrum.
Following Ardhuin et al. (2011) a better way to use Qs is the following

In [ ]:
period = False
Qf=(0.4+0.4*(1-np.tanh(15*(2*freqs-0.14))))*Q
plt.figure()
plt.title("Q vs Frequency from Q = {}".format(Q))
if period:
    plt.plot(1/freqs, Qf)
    plt.xlabel("Period (s)")
else:
    plt.plot(freqs, Qf)
    plt.xlabel("Frequency")
plt.ylabel("Q")
plt.show()

# Now lets compute the spectra for a range of values of Q

The results are saved in the folder "DATA/Q" for later use

We will plot them in the next notebook

In [ ]:
#Qs = np.linspace(150,500,36)
Qs = np.linspace(250,500,6)
for Q in Qs:
    dfF_fs = get_ww3(configs, Q, 10, lats, lons, Re, dpt, Cf, distance_df, plot=False)